### General purpose data cleaning. 
#### If you have cleaned a file let's keep track of what we did here.

In [1]:
import pandas as pd
import numpy as np
import geocoder
import requests
import time
from geopy.geocoders import Nominatim
import json 
from sqlalchemy.sql import text
import boto3

from File_Utilities import FileTools
import DB_Utilities
DBTools = DB_Utilities.DBTools()  # instantiate the class
FileTools.MYDIR = "./data/"

### Making crosswalk for: merging on city name and state name to fips codes.

#### Write some methods to use a geocoder to get the results and examine them in a more systematic way.

In [2]:
client = boto3.client('location')
geolocator = Nominatim(user_agent="UVaPatent22Capstone_DataScience")
null_response = {'original_state' : "",'original_city' : "",'county_GEOID' : "",'county_fip' :"", 'county_BASENAME' :"",'state_fip' :"",'state_BASENAME' : "",'state_STUSAB' : "",'city_BASENAME' : "", 'city_NAME' : "",  'status' : "" }


def get_coordinates_aws(city, state):    
    # THis costs money...
    try:
        result = client.search_place_index_for_text(FilterCountries=['USA'], IndexName='City_State_lookup', Text=f'{city}, {state}', MaxResults=3)
        # print(f'The type of this variable is : {type(result)}')
        lat = result['Results'][0]['Place']['Geometry']['Point'][0]
        long = result['Results'][0]['Place']['Geometry']['Point'][1]
        return True, str(lat), str(long), result
    except:
        return False, "", "", {}


def get_coordinates_Nominatim(city, state):    
    # Nominatim requests that we only make 1 request per second, otherwise we might get blocked.
    try:
        # location = geolocator.geocode({'city':city, 'state':state})
        location = geolocator.geocode(f'{city} {state}', exactly_one=False, limit=3, addressdetails=True )
        # print(location)
        time.sleep(1)
        # return True, str(location.latitude), str(location.longitude), location      
        return True, str(location.longitude), str(location.latitude), location      
    except Exception as e:
        print(e)
        time.sleep(1)
        return False, "", "", {}

def get_county_information_from_census(lat, long, city, state):
    # print (f'Lat : {lat }, Long : {long}')
    url = f'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?x={lat}&y={long}&benchmark=4&vintage=4&format=json'
    form_url = f'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?x={lat}&y={long}&benchmark=4&vintage=4'
    # print(form_url)

# Probably should be its own class at this point.
    try:
        response = requests.get(url)        
        if(response.status_code != 200):
            raise        
        data = response.json()

        county_GEOID = data['result']['geographies']['Counties'][0]['GEOID']    
        county_fip = data['result']['geographies']['Counties'][0]['COUNTY']
        county_BASENAME = data['result']['geographies']['Counties'][0]['BASENAME']

        state_fip = data['result']['geographies']['States'][0]['STATE']
        state_BASENAME = data['result']['geographies']['States'][0]['BASENAME']
        state_STUSAB = data['result']['geographies']['States'][0]['STUSAB'] 

        city_BASENAME = data['result']['geographies']['County Subdivisions'][0]['BASENAME'] 
        city_NAME = data['result']['geographies']['County Subdivisions'][0]['NAME']  


    except:
        return null_response



    print(f'{city_BASENAME} , {state_BASENAME} , {county_BASENAME} , {county_GEOID}')
    census_object = {
        'original_state' : state,
        'original_city' : city,
        'county_GEOID' : county_GEOID ,
        'county_fip' :county_fip, 
        'county_BASENAME' :county_BASENAME ,
        'state_fip' :state_fip ,
        'state_BASENAME' : state_BASENAME ,
        'state_STUSAB' : state_STUSAB ,
        'city_BASENAME' : city_BASENAME, 
        'city_NAME' : city_NAME , 
        'form_url' : form_url,        
        'status' : state_STUSAB == state } 

    return census_object



def geocode_lat_long(lat, long, city="", state=""):
    # print("  ")
    # print(f'{city} , {state}')


    if DBTools.get_row_count("aws_lookup_cache", f"city = '{city}' and state = '{state}'") == 0: 
        print(f"Geocoding {Lat}, {long}")      
        census_lookup_result = get_county_information_from_census(lat, long, city, state)
        DBTools.insert_location_lookup_cache(city = city
                                        , state = state
                                        , geocode_response = {}
                                        , census_lookup_result = census_lookup_result
                                        , lat = lat
                                        , long = long
                                        )

        return census_lookup_result

    else:
        print(f"{city}, {state} already exists in the cache")
        return null_response


def geocode_city_state(city, state, geocoder_service='aws'):
    # print("  ")
    # print(f'{city} , {state}')


    if DBTools.get_row_count("aws_lookup_cache", f"city = '{city}' and state = '{state}'") == 0: 
        print(f"Geocoding {city}, {state}")      
        if(geocoder_service == 'aws'):
            geocode_success, lat, long, geocode_response = get_coordinates_aws(city, state)
        else:
            geocode_success, lat, long, geocode_response = get_coordinates_Nominatim(city, state)

        if (geocode_success == False):
            return null_response
        else:

            census_lookup_result = get_county_information_from_census(lat, long, city, state)
            DBTools.insert_location_lookup_cache(city = city
                                            , state = state
                                            , geocode_response = geocode_response
                                            , census_lookup_result = census_lookup_result
                                            , lat = lat
                                            , long = long
                                            )

        return census_lookup_result

    else:
        print(f"{city}, {state} already exists in the cache")
        return null_response

        
    # To see the results in an html form.
    # https://geocoding.geo.census.gov/geocoder/geographies/coordinates?x=-70.1441014&y=44.671539&benchmark=4&vintage=4
    # https://geocoding.geo.census.gov/geocoder/geographies/coordinates?x=-89.7134&y=45.6038&benchmark=4&vintage=4
    	

# census_lookup_result = geocode_city_state('Parkridge','NJ', geocoder_service='aws')
# using Nominatim geocoder - it doesn't do a good a job of fuzzy matching. There might be a way to get it to match better...but it's also free.
# census_lookup_result = geocode_city_state('Parkridge','NJ', geocoder_service='Nominatim')

In [3]:

# Testing the geocoder, could write this into a unit test.
test = False
if test:
    null_response = {'original_state' : "",'original_city' : "",'county_GEOID' : "",'county_fip' :"", 'county_BASENAME' :"",'state_fip' :"",'state_BASENAME' : "",'state_STUSAB' : "",'city_BASENAME' : "", 'city_NAME' : "",  'status' : "" }
    test_list = [
        {'city' : 'city', 'state' : 'state', "geocode_response" : {"something" : "something"} , "census_lookup_result" : null_response}, 
        {'city' : 'city1', 'state' : 'state1', "geocode_response" : {"something" : "something"} , "census_lookup_result" : null_response}, 
        {'city' : 'city2', 'state' : 'state2', "geocode_response" : {"something" : "something"} , "census_lookup_result" : null_response}, 
        {'city' : 'city3', 'state' : 'state3', "geocode_response" : {"something" : "something"} , "census_lookup_result" : null_response}]

    for i in test_list:
        city = i['city']
        state = i['state']
        geocode_response = i['geocode_response']
        census_lookup_result = i['census_lookup_result']
        DBTools.insert_location_lookup_cache(city=city, state=state, geocode_response=geocode_response, census_lookup_result=census_lookup_result, lat="123.456", long="456.789")


In [9]:
# All the locations that we need from our data work
pregrant_locations_file = r"./data/pregrant/pregrant_locations.csv"
_pregrant_locations_df = pd.read_csv(pregrant_locations_file)

# All the locations
location_file = r"./data/pregrant/location.tsv"
_location_df = pd.read_csv(location_file, sep='\t')

# Seriously, these leading zeros are annoying.
_location_df[['state_fips', 'county_fips']] = _location_df[['state_fips','county_fips']].fillna("")
_location_df.loc[_location_df.state_fips!="", 'state_fips'] = _location_df.loc[_location_df.state_fips!="", 'state_fips'].astype(str).str.replace("\.0", "").str.zfill(2)
_location_df.loc[_location_df.county_fips!="", 'county_fips'] = _location_df.loc[_location_df.county_fips!="", 'county_fips'].astype(str).str.replace("\.0", "").str.zfill(5)
DBTools.insert_df(_location_df, "pregrant_location_unfiltered")

# _location_df.query("county_fips != ''").sort_values(by=['county_fips'], ascending=True)

C:\Users\micha\AppData\Local\Temp\ipykernel_29548\4205843355.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  _location_df.loc[_location_df.state_fips!="", 'state_fips'] = _location_df.loc[_location_df.state_fips!="", 'state_fips'].astype(str).str.replace("\.0", "").str.zfill(2)
C:\Users\micha\AppData\Local\Temp\ipykernel_29548\4205843355.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  _location_df.loc[_location_df.county_fips!="", 'county_fips'] = _location_df.loc[_location_df.county_fips!="", 'county_fips'].astype(str).str.replace("\.0", "").str.zfill(5)


In [5]:
# We are only interested in the locations that are in the pregrant locations file that have missing data.
reduced_location_df = pd.merge(_pregrant_locations_df, _location_df, left_on=['location_id'], right_on=['id'], how='left')
print(reduced_location_df.shape)
reduced_location_df = reduced_location_df.query("country == 'US' & county_fips == '' & city != '' & state != '' ", engine="python")
print(reduced_location_df.shape)

(164405, 11)
(23176, 11)


In [8]:
_location_df.query("id=='baa6fcdc-cb8e-11eb-9615-121df0c29c1e'")
# _location_df.head()

,id,city,state,country,latitude,longitude,county,state_fips,county_fips
178214,baa6fcdc-cb8e-11eb-9615-121df0c29c1e,Vers,NaN,FR,46.8221,5.9097,NaN,,


In [ ]:
# We need all the data to join back once we have geocoded the locations.
reinsert = False
if reinsert:
    pregrant_location_all_df = pd.merge(_pregrant_locations_df, _location_df, left_on=['location_id'], right_on=['id'], how='left')
    DBTools.insert_df(pregrant_location_all_df, "pregrant_location_all")

In [ ]:
# Actually run the geocoder and persist the results.
# We need to geta dataframe from the database for the final results.
rerun = False
if rerun:
    reduced_location_df.apply(lambda x: geocode_city_state(x.city, x.state), axis=1, result_type='expand')

In [ ]:
location_crosswalk_df = DBTools.get_df("pregrant_location_crosswalk", "GEOID != ''")
location_crosswalk_df.shape


In [ ]:
# save the results to a file

# I'm adding in a dummy row so the GEOIDS are string and not numeric. It won't match on anything else.
location_crosswalk_df_dummy = location_crosswalk_df.head(1).copy()
location_crosswalk_df_dummy['id'] = "_"
location_crosswalk_df_dummy['GEOID'] = "_"
location_crosswalk_df = pd.concat([location_crosswalk_df, location_crosswalk_df_dummy])


location_crosswalk_df.to_csv("./data/pregrant/location_crosswalk.csv", index=False)


In [ ]:
# load a csv into dataframe
reload = pd.read_csv("./data/pregrant/location_crosswalk.csv", low_memory=False)
reload.shape

## County Business Patterns 
### Pivoting tables so county is row and all predictors are columns

In [ ]:
# Load in the data.
_cbp_lq_3digit_naics_all_years_df = FileTools.load_df_from_parquet('cbp_lq_3digit_naics_all_years.gzip')
_cbp_lq_3digit_naics_all_years_df.columns


In [ ]:
all_data_cbp = pd.DataFrame()
for i in range(2010,2021): 
# for i in range(2020,2021): 
    year = str(i)
    print(f'Working on year : {year}')    

    # Filter by the year we want.
    temp_cbp_lq_3digit_naics_all_years_df = _cbp_lq_3digit_naics_all_years_df[
        (_cbp_lq_3digit_naics_all_years_df['year'] == year) 
        & (_cbp_lq_3digit_naics_all_years_df['naics_level'] == 3)
        ][[
            'county_fips','qp1', 'ap', 'est','naics','location_quotient_county_state','location_quotient_county_us']].copy()


    # Get some aggregations by county
    group_cbp_lq_3digit_naics_all_years_df = temp_cbp_lq_3digit_naics_all_years_df[['county_fips','qp1', 'ap', 'est']].groupby(['county_fips']).sum().reset_index()
    
    
    # Pivot all the naics codes up into columns
    pivot_cbp_lq_2digit_naics_all_years_df = pd.pivot_table(temp_cbp_lq_3digit_naics_all_years_df, values='location_quotient_county_us', index=['county_fips'],
                        columns=['naics'], aggfunc=np.average, fill_value=0)


    # Add the aggregations to the pivot table
    merge_cbp_lq_2digit_naics_all_years_df = pivot_cbp_lq_2digit_naics_all_years_df.merge(group_cbp_lq_3digit_naics_all_years_df, on='county_fips', how='left')
    merge_cbp_lq_2digit_naics_all_years_df['year'] = year

    # Do I need to do this?
    merge_cbp_lq_2digit_naics_all_years_df.reset_index(inplace=True)
    all_data_cbp = pd.concat([all_data_cbp, merge_cbp_lq_2digit_naics_all_years_df])

FileTools.save_df_as_parquet(all_data_cbp, 'cbp_lq_3digit_naics_all_years_PIVOT.gzip')
FileTools.save_df_as_csv(all_data_cbp, 'cbp_lq_3digit_naics_all_years_PIVOT.csv')    

In [ ]:
all_data_cbp.head()

In [13]:
all_data_cbp = FileTools.load_df_from_parquet('cbp_lq_3digit_naics_all_years_PIVOT.gzip')
print(all_data_cbp.shape)
all_data_cbp.head()
# DBTools.truncate_and_insert_df(all_data_cbp, "cbp_PIVOT")

(34279, 92)


,index,county_fips,113,114,115,211,212,213,221,236,...,713,721,722,811,812,813,qp1,ap,est,year
0,0,01001,5.555412,0.000000,0.000000,0.0,1.945867,0.0,3.227681,0.681620,...,1.281011,1.475549,2.406618,0.915081,1.075647,2.585839,40430,174710,870,2010
1,1,01003,0.000000,0.285288,0.000000,0.0,0.000000,0.0,0.000000,1.066502,...,1.313262,1.722986,1.721987,0.971572,0.756357,1.509355,288364,1346818,4656,2010
2,2,01005,31.637423,0.000000,6.209217,0.0,0.000000,0.0,0.000000,0.000000,...,0.000000,0.590252,0.721443,0.509799,0.000000,0.655996,19058,83058,514,2010
3,3,01007,50.045967,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.822521,1.026759,1.284786,1.061995,9317,40640,292,2010
4,4,01009,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,1.002452,...,0.610964,0.000000,0.887612,2.018704,1.447656,1.658862,26070,113977,699,2010


##

### Working on Economic Indicators

* CAINC4__ALL_AREAS_1969_2020.csv
* https://virginia.box.com/s/v70g9niz9att0381tapsnmtffksq7ahw

In [10]:
# Read in the data, there are some encoding issues so we need to explicitly specify the encoding.
_CAINC4__ALL_AREAS_1969_2020_df = pd.read_csv(FileTools.get_full_file_path('CAINC4__ALL_AREAS_1969_2020.csv'),low_memory=False, encoding = "ISO-8859-1" )

# Clean up the description field a bit.
_CAINC4__ALL_AREAS_1969_2020_df['Description'] = _CAINC4__ALL_AREAS_1969_2020_df.Description.str.strip().str.replace("/","")

# Seems like there is double data for 'Employer contributions for government social insurance' but they have different LineCodes so we can take out one of them.
# The other descriptions are in 30's and on so we will take out 62.
_CAINC4__ALL_AREAS_1969_2020_df = _CAINC4__ALL_AREAS_1969_2020_df[~(_CAINC4__ALL_AREAS_1969_2020_df.LineCode == 62)]
# DBTools.truncate_and_insert_df(_CAINC4__ALL_AREAS_1969_2020_df, "CAINC4__ALL_AREAS_1969_2020_df")

### Pivot the table so that county is row and all predictors are columns by year

In [11]:
all_data_ei = pd.DataFrame()
for i in range(1969,2021):  
# for i in range(1969,1970):
    year = str(i)
    print(f'Working on year : {year}')
    temp_CAINC4__ALL_AREAS_1969_2020_df = _CAINC4__ALL_AREAS_1969_2020_df[[year, 'GeoFIPS',  'Description']].copy()
    pivot_CAINC4__ALL_AREAS_1969_2020_df = pd.pivot_table(temp_CAINC4__ALL_AREAS_1969_2020_df, 
                                                    values=year
                                                    ,index=['GeoFIPS']
                                                    ,columns=['Description']
                                                    ,aggfunc=np.sum, fill_value=0)

    pivot_CAINC4__ALL_AREAS_1969_2020_df['year'] = year
    pivot_CAINC4__ALL_AREAS_1969_2020_df.reset_index(inplace=True)
    all_data_ei = pd.concat([all_data_ei, pivot_CAINC4__ALL_AREAS_1969_2020_df])   

all_data_ei.GeoFIPS = all_data_ei.GeoFIPS.str.replace('"','')

FileTools.save_df_as_parquet(all_data_ei, 'CAINC4_ALL_AREAS_1969_2020_PIVOT.gzip')
FileTools.save_df_as_csv(all_data_ei, 'CAINC4_ALL_AREAS_1969_2020_PIVOT.csv')
# DBTools.truncate_and_insert_df(all_data_ei, "CAINC4__ALL_AREAS_1969_2020_PIVOT")

Working on year : 1969
Working on year : 1970
Working on year : 1971
Working on year : 1972
Working on year : 1973
Working on year : 1974
Working on year : 1975
Working on year : 1976
Working on year : 1977
Working on year : 1978
Working on year : 1979
Working on year : 1980
Working on year : 1981
Working on year : 1982
Working on year : 1983
Working on year : 1984
Working on year : 1985
Working on year : 1986
Working on year : 1987
Working on year : 1988
Working on year : 1989
Working on year : 1990
Working on year : 1991
Working on year : 1992
Working on year : 1993
Working on year : 1994
Working on year : 1995
Working on year : 1996
Working on year : 1997
Working on year : 1998
Working on year : 1999
Working on year : 2000
Working on year : 2001
Working on year : 2002
Working on year : 2003
Working on year : 2004
Working on year : 2005
Working on year : 2006
Working on year : 2007
Working on year : 2008
Working on year : 2009
Working on year : 2010
Working on year : 2011
Working on 

'CAINC4_ALL_AREAS_1969_2020_PIVOT.csv'

In [12]:
all_data_ei.head()

Description,GeoFIPS,Earnings by place of work,Employee and self-employed contributions for government social insurance,Employer contributions for employee pension and insurance funds 8,Employer contributions for government social insurance,Equals: Net earnings by place of residence,Farm income 2,Farm proprietors' income,Less: Contributions for government social insurance 5,Nonfarm personal income 1,...,"Plus: Dividends, interest, and rent 7",Plus: Personal current transfer receipts,Population (persons) 3,Proprietors employment,Proprietors' income 9,Supplements to wages and salaries,Total employment,Wage and salary employment,Wages and salaries,year
0,00000,653665000,21204000,41314000,22587000,609767000,17275000,13407000,43791000,773954000,...,119140000,62322000,201298000,12331200,77578000,63901000,91053200,78722000,512186000,1969
1,01000,8112293,278249,538937,272228,7692073,277960,230494,550477,9459755,...,1153840,891802,3440000,195784,953312,811165,1411234,1215450,6347816,1969
2,01001,33797,1173,1463,990,56846,3544,2647,2163,66429,...,7626,5501,25166,1173,8895,2453,6630,5457,22449,1969
3,01003,80410,3011,3358,2605,118705,6504,4501,5616,150687,...,23653,14833,56951,5122,16981,5963,20008,14886,57466,1969
4,01005,36403,1318,1452,1142,38372,3977,3410,2460,47153,...,6010,6748,23818,2279,9442,2594,9428,7149,24367,1969


In [ ]:
# test = FileTools.load_df_from_csv('CAINC4_ALL_AREAS_1969_2020_PIVOT.csv')
test = pd.read_csv(FileTools.get_full_file_path('CAINC4_ALL_AREAS_1969_2020_PIVOT.csv'),low_memory=False, encoding = "ISO-8859-1" )
test.head()